In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import torch
batch_size = 10

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'

In [ ]:
def onehost_encoder(position, depth):
    onehot = torch.zeros(depth)
    onehot[position] = 1
    return onehot

In [ ]:
def int_to_onehot(number):
    return onehost_encoder(number, 100)

In [ ]:
sample = int_to_onehot(75)

In [ ]:
def onehot_to_int(onehot):
    return torch.argmax(onehot).item()

In [ ]:
onehot_to_int(sample)

In [ ]:
def gen_squence():
    indices = torch.randint(0, 20, (batch_size,))
    return indices * 5

In [ ]:
gen_squence()

In [ ]:
def gen_batch():
    sequence = gen_squence()
    batch = [int_to_onehot(i).numpy() for i in sequence]
    batch = np.array(batch)
    return torch.tensor(batch)

In [ ]:
batch = gen_batch()

In [ ]:
batch

In [ ]:
def data_to_num(data):
    return torch.argmax(data, dim=-1)

In [ ]:
numbers = data_to_num(batch)

In [ ]:
numbers

In [ ]:
from torch import nn

In [ ]:
D = nn.Sequential(
    nn.Linear(100, 1),
    nn.Sigmoid()
).to(device)

In [ ]:
G = nn.Sequential(
    nn.Linear(100, 100),
    nn.ReLU()
).to(device)

In [ ]:
loss_fn = nn.BCELoss()
lr = .0005
optimD = torch.optim.Adam(D.parameters(), lr=lr)
optimG = torch.optim.Adam(G.parameters(), lr=lr)
fake_labels = torch.ones(batch_size, 1).to(device)
real_labels = torch.zeros(batch_size, 1).to(device)

In [ ]:
def train_D_on_real(dis_model, dis_optim, loss_fn):
    real_data = gen_batch().to(device)
    optimD.zero_grad()
    pred = dis_model(real_data)
    loss = loss_fn(pred, real_labels)
    loss.backward()
    dis_optim.step()
    return loss.item()

def train_D_on_fake(dis_model, dis_optim, loss_fn, fake_samples):
    dis_optim.zero_grad()
    pred = dis_model(fake_samples)
    loss = loss_fn(pred, fake_labels)
    loss.backward()
    dis_optim.step()
    return loss.item()

def train_G(g_model, d_model, g_optim, loss_fn):
    g_optim.zero_grad()
    noise = torch.randn(batch_size, 100).to(device)
    generated_data = g_model(noise)
    output = d_model(generated_data)
    loss = loss_fn(output, real_labels)
    loss.backward()
    g_optim.step()
    return generated_data


In [ ]:
class EarlyStop:
    def __init__(self, patience=1000):
        self.patience = patience
        self.steps = 0
        self.min_gdif = float('inf')
    def stop(self, gdif):
        if gdif < self.min_gdif:
            self.min_gdif = gdif
            self.steps = 0
        elif gdif >= self.min_gdif:
            self.steps += 1
        if self.steps >= self.patience:
            return True
        else:
            return False
stopper=EarlyStop(800)

In [ ]:
mse = nn.MSELoss()

In [ ]:
def distance(generated_data):
    nums = data_to_num(generated_data)
    remainders = nums % 5
    ten_zeros = torch.zeros_like(remainders, dtype=torch.float32).to(device)
    mseloss = mse(remainders, ten_zeros)
    return mseloss

In [ ]:
def train_D_G(d_model, g_model, loss_fn, optim_d, optim_g):
    train_D_on_real(d_model, optim_d, loss_fn)
    fake_samples = g_model(torch.randn(batch_size, 100).to(device))
    train_D_on_fake(d_model, optim_d, loss_fn, fake_samples)
    return train_G(g_model, d_model, optim_g, loss_fn)



In [ ]:
for epoch in range(10000):
    generated_data = train_D_G(D, G, loss_fn, optimD, optimG)
    dis = distance(generated_data)
    if stopper.stop(dis):
        break
    if epoch % 50 == 0:
        print(data_to_num(generated_data))    #D

In [ ]:
torch.jit.save(torch.jit.script(G), 'files/nums_gen.pt')